In [1]:
import numpy as np
import pandas as pd

In [2]:
import prepostprocessing.cleaning as cleaning
import prepostprocessing.pre_processing as preproc
from qapf import qapf
from qapf import cipw

___

# CIPWFULL normalized data

## Convert to CIPWFULL format

**I used an arbitrary version of the mineralogical data, not sure if this is the correct one** -->is a good one

In [3]:
mineralogy_original = pd.read_excel("../_CIPW/CIPW/AREA1_J/interpolated_data.xlsx", index_col=0)
# Only iterested in sample number and oxides
mineralogy = mineralogy_original[['SiO2', 'TiO2', 'Al2O3', 'Fe2O3','FeO', 'MnO', 'MgO', 'CaO', 'Na2O', 'K2O', 'oth']]
mineralogy.columns

Index(['SiO2', 'TiO2', 'Al2O3', 'Fe2O3', 'FeO', 'MnO', 'MgO', 'CaO', 'Na2O',
       'K2O', 'oth'],
      dtype='object')

In [4]:
mineralogy = mineralogy.dropna()

In [5]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [6]:
# Remove 'l.i.'' and 'oth' columns and immediately normalize data
mineralogy = preproc.normalize(mineralogy.drop(["oth"], axis=1))

In [7]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [10]:
mineralogy.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
2822,72.408050,0.254845,14.662159,0.675796,1.924432,0.042224,0.707242,1.452687,3.809990,4.062576
2823,72.408664,0.254826,14.661993,0.675956,1.924167,0.042222,0.707109,1.452401,3.809979,4.062685
2824,71.783621,0.280009,14.831792,0.625671,2.114012,0.044135,0.838003,1.705294,3.818477,3.958985
2825,71.783836,0.280002,14.831743,0.625707,2.113931,0.044135,0.837956,1.705196,3.818475,3.959019
2826,71.782975,0.280037,14.831960,0.625654,2.114182,0.044137,0.838141,1.705548,3.818484,3.958882


In [11]:
mineralogy

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
2822,72.408050,0.254845,14.662159,0.675796,1.924432,0.042224,0.707242,1.452687,3.809990,4.062576
2823,72.408664,0.254826,14.661993,0.675956,1.924167,0.042222,0.707109,1.452401,3.809979,4.062685
2824,71.783621,0.280009,14.831792,0.625671,2.114012,0.044135,0.838003,1.705294,3.818477,3.958985
2825,71.783836,0.280002,14.831743,0.625707,2.113931,0.044135,0.837956,1.705196,3.818475,3.959019
2826,71.782975,0.280037,14.831960,0.625654,2.114182,0.044137,0.838141,1.705548,3.818484,3.958882
...,...,...,...,...,...,...,...,...,...,...
184622,71.860232,0.292230,14.846404,0.920042,1.897721,0.042010,0.795073,1.552240,3.816377,3.977669
184623,71.811710,0.294749,14.860538,0.922414,1.907475,0.042113,0.805078,1.569103,3.817015,3.969804
184624,71.755945,0.297646,14.876611,0.925008,1.918732,0.042232,0.816665,1.588623,3.817724,3.960816
184625,71.692284,0.300957,14.894737,0.927836,1.931620,0.042367,0.830005,1.611072,3.818501,3.950621


In [13]:
mineralogy1 = mineralogy.iloc[0:45000]
#mineralogy2 = mineralogy.iloc[45001:90000]

In [14]:
cipw.convert_to_CIPWFULL_format(mineralogy1, "../_CIPW/CIPW/AREA1_J/area1_J_1.txt", dataset_name="Vistelius (1995)")
#cipw.convert_to_CIPWFULL_format(mineralogy2, "../_CIPW/CIPW/AREA1/area1_2.txt", dataset_name="Vistelius (1995)")

_____

## Extract results from CIPWFULL results file

**I only selected quartz, plagioclase and alkali feldspar here as they are of interest to us for the QAPF classification**  
**Feel free to extract addiotnal minerals if they would be of any interest to you**

**Not sure what to do with albite in the final assignments as by the book it should be accounted for as an alkali feldspar and thus be included in the sum of plagioclase but then the resulting plagioclase contents seem quite low.   
This is something you might look into although the method used here seems to be correct according to te resulting QAPF diagram.**

--> indeed when reading the paper of Streckeisen (and other sources), albite should be accounted for as an alkali feldspar, also saw that the datapoints were generally located more to the right side (plagioclase), maybe an overestimation of plagioclase?

--> I changed the settings below so that albite belongs to the alkali feldspars, Think this figure should be more correct according to the papers, but the cotent of plagioclase is indeed quite low. 

In [17]:
df_final = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA1_J/area1_J_1_results_excel.txt")
#df_final2 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA1/area1_2_results_excel.txt")

In [18]:
df_final

,Q,P,K
SAMPLE,,,
2822,29.329699,36.219552,27.229990
2823,29.331070,36.218037,27.230571
2824,28.156086,37.537296,26.624987
2825,28.156486,37.536850,26.625222
2826,28.154932,37.538586,26.624396
...,...,...,...
184622,28.867996,36.762059,26.733652
184623,28.789419,36.850494,26.687682
184624,28.698441,36.952735,26.635218


## QAPF classification

In [19]:
df_final["QAPF"] = qapf.check_QAPF_region(df_final)

In [20]:
df_final["QAPF"].unique()

array(['monzo granite', 'granodiorite',
       'quartz monzodiorite\nquartz monzogabbro'], dtype=object)

In [21]:
df_final.to_excel("../_CIPW/CIPW/AREA1_J/qapf.xlsx", index=True)

-------

In [22]:
df_final.iloc[:, 3].value_counts().to_excel("../_CIPW/CIPW/AREA1_J/QAPF_counts.xlsx")

-----

In [23]:
QAPF_control = pd.read_excel("../_CIPW/CIPW/1_normalized.xlsx", index_col=0)
Area = pd.read_excel("../_INTERPOLATION/area1_J.xlsx", index_col=0, usecols = [0, 1, 2, 3, 4, 29])

In [24]:
QAPF_control = pd.concat([Area, QAPF_control], axis = 1)
QAPF_control = QAPF_control.dropna()
QAPF_control
QAPF_control.to_excel("../_CIPW/CIPW/AREA1_J/QAPF_control_points.xlsx", index=True)